In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
from nanodrz.model import DiarizeGPT, Config
from nanodrz import data
from nanodrz.data import artificial_diarisation_sample
from nanodrz.utils import visualise_annotation, play
from nanodrz.download import dl_scp_file
from glob import glob
from os import path
import json
from nanodrz.constants import CACHE_DIR
from nanodrz import format_conversions as format

device = torch.device("cuda:0")
torch.cuda.set_device(device)
# https://drive.google.com/file/d/1Cg_GPHxpcWMqGpsDlLMXlr5ui0mPYacn/view?usp=sharing
ckpt = torch.load("/home/harry/runs/nanodrz/1708904995/0014000.pt", map_location=device)
model:DiarizeGPT = DiarizeGPT.from_pretrained(ckpt).cuda()

In [ ]:
# Use the same parameters that the model was trained on to generate a sample
for i in range(3):
    audio, labels = artificial_diarisation_sample(data.get_speakers(), min_secs=10, num_speakers=2, max_secs=15)
    play(audio)
    reference = format.labels_to_annotation(labels)

    nlabels = model.generate(audio.cuda(), temperature=.3, max_steps=(len(labels))*3)

    hypothesis = format.labels_to_annotation(nlabels)
    visualise_annotation(labels)
    visualise_annotation(nlabels)

    from pyannote.metrics.diarization import DiarizationErrorRate

    metric = DiarizationErrorRate()
    print(metric(reference, hypothesis))